In [1]:
import pandas as pd
import torch 

df=pd.read_csv("train_clean_kalman.csv")
df['signal']=df['signal']/df['signal'].max()

In [2]:
df['open_channels'].max()

10

# Create time series dataset

In [3]:
class TimesereisDataset(torch.utils.data.Dataset):
    def __init__(self, df:pd.Series(), seqLen=10):
        super().__init__()
        self.df=df
        self.seqLen=seqLen
    
    def __len__(self):
        return self.df.shape[0]-self.seqLen-1
    
    def __getitem__(self,index):
        x=self.df.iloc[index:index+self.seqLen,1].values
        y=self.df.iloc[index+self.seqLen-1,2] 
        return x,y   

In [4]:
seqLen=50
timeSeriesDataset = TimesereisDataset(df,seqLen)

In [5]:
trainNumbers=int(len(timeSeriesDataset)*0.9)
trainDataset,testDataset=torch.utils.data.random_split(timeSeriesDataset,[trainNumbers,len(timeSeriesDataset)-trainNumbers])

In [6]:
trainDataLoader=torch.utils.data.DataLoader(trainDataset,batch_size=8,shuffle=True)
testDataLoader=torch.utils.data.DataLoader(testDataset,batch_size=8,shuffle=True)

In [7]:
from Wavenet import WaveNet,WaveNetClassifier
from tqdm import tqdm 

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device="cpu"
wavenetClassifierModel=WaveNetClassifier(seqLen,df['open_channels'].max()+1)
wavenetClassifierModel.to(device)

wavenetClassifierModel.train()

optimizer=torch.optim.AdamW(wavenetClassifierModel.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
lossFunction = torch.nn.CrossEntropyLoss()

In [8]:
def calc_accuracy(Out,Y):
    max_vals, max_indices = torch.max(Out,1)
    train_acc = (max_indices == Y).sum().item()/max_indices.size()[0]
    return train_acc

In [9]:
epochs=1
globalStep=500

for epoch in range(epochs):
    for step, (x_train,y_train) in tqdm(enumerate(trainDataLoader),desc="Training"):
         x_train = torch.unsqueeze(x_train,dim=1).float()
            
         x_train.to(device)
         y_train.to(device)
            
         output = wavenetClassifierModel(x_train)
         output = torch.squeeze(output,dim=1)

         loss= lossFunction(output,y_train)
         optimizer.zero_grad()
            
         loss.backward()
         optimizer.step()
            
         if step % globalStep == 0:
            
            # scheduler.step()
            # print(output.detach().numpy())
            # print(y_train.numpy())
            
            with torch.no_grad():
                accuracy=0
                loss=0
                
                for stepTest, (x_test,y_test) in tqdm(enumerate(testDataLoader),desc="Validation"):
                    x_test.to(device)
                    y_test.to(device)
                    
                    x_test = torch.unsqueeze(x_test,dim=1).float()
                    
                    output=wavenetClassifierModel(x_test)
                    output = torch.squeeze(output,dim=1)
                    
                    accuracy+=calc_accuracy(output,y_test)*100
                    loss+= lossFunction(output,y_test).item()
                    
                    if stepTest>200:
                        break
                        
            print(f"loss for step {step} : {loss/stepTest}  :  Accuracy: {accuracy/stepTest} %")
         
    print(f"epoch {epoch}")

Training: 0it [00:00, ?it/s]/home/adhiraj/miniconda3/envs/wavenet/lib/python3.10/site-packages/torch/nn/modules/conv.py:309: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /opt/conda/conda-bld/pytorch_1682343967769/work/aten/src/ATen/native/Convolution.cpp:1003.)
  return F.conv1d(input, weight, bias, self.stride,

Validation: 0it [00:00, ?it/s]
Validation: 4it [00:00, 39.55it/s]
Validation: 17it [00:00, 88.53it/s]
Validation: 29it [00:00, 102.47it/s]
Validation: 41it [00:00, 107.58it/s]
Validation: 52it [00:00, 107.52it/s]
Validation: 63it [00:00, 91.50it/s] 
Validation: 73it [00:00, 86.00it/s]
Validation: 84it [00:00, 90.37it/s]
Validation: 95it [00:01, 94.16it/s]
Validation: 107it [00:01, 101.08it/s]
Validation: 119it [00:01, 104.31it/s]
Validation: 130it [00:01, 105.14it/s]
Validation: 141it [00:01, 88.07it/s] 
Validation: 151it [00:01, 89.46it/s]
Validation: 161it [00:01, 8

loss for step 0 : 2.4244583080064004  :  Accuracy: 3.3582089552238807 %


Training: 497it [00:15, 34.82it/s]
Validation: 0it [00:00, ?it/s]
Validation: 7it [00:00, 67.36it/s]
Validation: 16it [00:00, 76.29it/s]
Validation: 26it [00:00, 85.55it/s]
Validation: 37it [00:00, 91.89it/s]
Validation: 48it [00:00, 96.57it/s]
Validation: 59it [00:00, 99.86it/s]
Validation: 72it [00:00, 107.10it/s]
Validation: 83it [00:00, 105.07it/s]
Validation: 95it [00:00, 108.16it/s]
Validation: 106it [00:01, 104.54it/s]
Validation: 118it [00:01, 106.91it/s]
Validation: 131it [00:01, 111.73it/s]
Validation: 144it [00:01, 116.12it/s]
Validation: 156it [00:01, 113.64it/s]
Validation: 168it [00:01, 110.85it/s]
Validation: 180it [00:01, 106.59it/s]
Validation: 201it [00:01, 103.18it/s]
Training: 506it [00:17,  8.33it/s]

loss for step 500 : 2.305035905458441  :  Accuracy: 23.942786069651742 %


Training: 1000it [00:30, 40.59it/s]
Validation: 0it [00:00, ?it/s]
Validation: 10it [00:00, 96.22it/s]
Validation: 23it [00:00, 111.27it/s]
Validation: 35it [00:00, 99.94it/s] 
Validation: 46it [00:00, 98.77it/s]
Validation: 56it [00:00, 95.15it/s]
Validation: 67it [00:00, 99.58it/s]
Validation: 79it [00:00, 103.72it/s]
Validation: 92it [00:00, 110.30it/s]
Validation: 104it [00:00, 108.41it/s]
Validation: 117it [00:01, 112.87it/s]
Validation: 129it [00:01, 109.04it/s]
Validation: 140it [00:01, 100.81it/s]
Validation: 151it [00:01, 91.31it/s] 
Validation: 161it [00:01, 91.94it/s]
Validation: 172it [00:01, 95.64it/s]
Validation: 184it [00:01, 100.89it/s]
Validation: 201it [00:01, 101.11it/s]
Training: 1005it [00:32,  6.80it/s]

...

loss for step 61000 : 2.29391723841577  :  Accuracy: 23.880597014925375 %


Training: 61498it [29:51, 37.32it/s]
Validation: 0it [00:00, ?it/s]
Validation: 9it [00:00, 85.47it/s]
Validation: 21it [00:00, 104.55it/s]
Validation: 33it [00:00, 108.75it/s]
Validation: 44it [00:00, 89.16it/s] 
Validation: 54it [00:00, 81.91it/s]
Validation: 63it [00:00, 71.62it/s]
Validation: 73it [00:00, 78.58it/s]
Validation: 83it [00:00, 82.88it/s]
Validation: 93it [00:01, 87.36it/s]
Validation: 104it [00:01, 91.28it/s]
Validation: 115it [00:01, 94.11it/s]
Validation: 125it [00:01, 94.28it/s]
Validation: 137it [00:01, 100.70it/s]
Validation: 148it [00:01, 103.16it/s]
Validation: 159it [00:01, 99.54it/s] 
Validation: 170it [00:01, 101.77it/s]
Validation: 183it [00:01, 108.18it/s]
Validation: 201it [00:02, 91.92it/s] 
Training: 61506it [29:54,  7.36it/s]

loss for step 61500 : 2.297572692235311  :  Accuracy: 24.564676616915424 %


Training: 61674it [29:59, 34.28it/s]


KeyboardInterrupt: 